In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import json
import pickle
import logging

import pandas as pd
import numpy as np

from itext2kg import iText2KG
from itext2kg.utils import PubtatorProcessor
from langchain_ollama import ChatOllama, OllamaEmbeddings
from itext2kg.graph_integration import GraphIntegrator

/home/jovyan/miniconda3/envs/kg/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
URI = "neo4j+s://435eb4f0.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = "nqL3At3rptFpfRrBhg5Y1-7k78oHqVaW42WKtqVRnJo"

llm = ChatOllama(
    model="deepseek-r1:32b",
    temperature=0,
)

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [4]:
df1 = pd.read_csv("/home/jovyan/my_code/RAG/AD_pmid_results.csv")
pmid_list = list(df1['pmid'].astype(str))
pmid_list = pmid_list[:10]  
PUTATOR_PATH = "/home/jovyan/my_code/RAG/Data_v2/AD_pubtabor/"
JSON_PATH = "/home/jovyan/my_code/itext2kg/output_kg/AD"
OUTPUT_PATH = JSON_PATH

In [5]:
pmid_list

['39652311',
 '39890535',
 '39855320',
 '39589570',
 '39604278',
 '39596657',
 '39568323',
 '39657091',
 '39276711',
 '39431763']

In [6]:
PMID = pmid_list[1]  # 这个 PMID 提取出来全是散的节点，没有的关系，重新跑一下，看看问题在哪里了

In [7]:
with open(f"{PUTATOR_PATH}{PMID}.txt", "r") as f:
    text = f.readlines()

In [8]:
text

["39890535|t|The associations between fresh vegetable and fruit consumption and plasma and PET biomarkers in preclinical Alzheimer's disease: A cross-sectional and longitudinal study of Chinese population.\n",
 "39890535|a|BACKGROUND: The identification of the modifiable lifestyle factors including dietary habits in older adults of preclinical Alzheimer's disease (AD) and early effective interventions are of great importance. OBJECTIVES: We studied whether the consumption of fresh vegetables and fruits was different between cognitively unimpaired (CU) and cognitively impaired (CI) population and mainly investigated the associations between vegetable and fruit consumption and PET and plasma AD biomarkers in older CU adults with higher beta-amyloid (Abeta) burden. DESIGN, SETTING, AND PARTICIPANTS: Older adults with the age of 50-85 years were enrolled for a cross-sectional and longitudinal study. The groups depended on whether the participants were CU or CI. Partial participants whose h

In [9]:
pubtator_file = f"{PUTATOR_PATH}{PMID}.txt"
pubtator_process = PubtatorProcessor(pubtator_file, llm)
semantic_blocks = pubtator_process.block
properties_info = pubtator_process.properties_info
pubtator_info = pubtator_process.pubtator_info

2025-03-02 12:16:47,391 - INFO - Processing PubTator file: /home/jovyan/my_code/RAG/Data_v2/AD_pubtabor/39890535.txt
2025-03-02 12:16:47,393 - WARNING - Skipping malformed entity line: ['']


2025-03-02 12:16:49,439 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-02 12:17:15,205 - INFO - Abstract distilled successfully.
2025-03-02 12:17:15,206 - INFO - Adding Alzheimer's disease to the abstract
2025-03-02 12:17:15,207 - INFO - Adding AD to the abstract
2025-03-02 12:17:15,208 - INFO - Adding cognitively impaired to the abstract
2025-03-02 12:17:15,208 - INFO - Adding CI to the abstract
2025-03-02 12:17:15,209 - INFO - Adding cognitive decline to the abstract
2025-03-02 12:17:15,209 - INFO - Adding [{'gene': 'Abeta'}, {'gene': 'tau'}, {'gene': 'Tau'}] to the abstract
2025-03-02 12:17:15,209 - INFO - Adding [{'chemical': '18F-florbetapir'}, {'chemical': '18F-MK6240'}] to the abstract


In [10]:
properties_info

{'source': 'PMID39890535'}

In [11]:
semantic_blocks

['disease - [[\'disease\': "Alzheimer\'s disease"], [\'disease\': "Alzheimer\'s disease"], [\'disease\': \'AD\'], [\'disease\': \'cognitively impaired\'], [\'disease\': \'CI\'], [\'disease\': \'cognitive decline\']]',
 "protein - [['protein': 'Tau']]",
 "gene - [['gene': 'Abeta'], ['gene': 'tau'], ['gene': 'Tau']]",
 "chemical - [['chemical': '18F-florbetapir'], ['chemical': '18F-MK6240']]",
 "Title: The associations between fresh vegetable and fruit consumption and plasma and PET biomarkers in preclinical Alzheimer's disease: A cross-sectional and longitudinal study of Chinese population. Abstract: BACKGROUND: The identification of the modifiable lifestyle factors including dietary habits in older adults of preclinical Alzheimer's disease (AD) and early effective interventions are of great importance. OBJECTIVES: We studied whether the consumption of fresh vegetables and fruits was different between cognitively unimpaired (CU) and cognitively impaired (CI) population and mainly invest

In [12]:
pubtator_info

{"Alzheimer's disease": {'label': 'disease', 'unique_id': 'MESH:D000544'},
 'AD': {'label': 'disease', 'unique_id': 'MESH:D000544'},
 'cognitively impaired': {'label': 'disease', 'unique_id': 'MESH:D003072'},
 'CI': {'label': 'disease', 'unique_id': 'MESH:D003072'},
 'Abeta': {'label': 'gene', 'unique_id': 'Gene ID:351'},
 '18F-florbetapir': {'label': 'chemical', 'unique_id': 'MESH:C545186'},
 '18F-MK6240': {'label': 'chemical', 'unique_id': '-'},
 'tau': {'label': 'gene', 'unique_id': 'Gene ID:4137'},
 'Tau': {'label': 'gene', 'unique_id': 'Gene ID:4137'},
 'cognitive decline': {'label': 'disease', 'unique_id': 'MESH:D003072'}}

In [13]:
properties_info

{'source': 'PMID39890535'}

In [14]:
# -----------------Build the graph-----------------
itext2kg = iText2KG(llm_model = llm, embeddings_model = embeddings)
kg = itext2kg.build_graph(
    sections=[semantic_blocks], 
    source=properties_info,  
    entities_info=pubtator_info,
    ent_threshold=0.9, 
    rel_threshold=0.4,
    context = semantic_blocks[-1],
    source_id = properties_info['source'] ,
    )

2025-03-02 12:17:15,349 - INFO - Extracting Entities from the Document


2025-03-02 12:17:16,529 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-02 12:17:46,736 - INFO - INFO --- alzheimer's disease add unique_id and label
2025-03-02 12:17:46,737 - INFO - INFO --- ad add unique_id and label
2025-03-02 12:17:46,738 - INFO - INFO --- cognitively impaired add unique_id and label
2025-03-02 12:17:46,739 - INFO - INFO --- ci add unique_id and label
2025-03-02 12:17:46,741 - INFO - INFO --- cognitive decline add unique_id and label
2025-03-02 12:17:46,741 - INFO - INFO --- tau add unique_id and label
2025-03-02 12:17:46,743 - INFO - INFO --- abeta add unique_id and label
2025-03-02 12:17:46,743 - INFO - INFO --- tau add unique_id and label
2025-03-02 12:17:46,744 - INFO - INFO --- 18f-florbetapir add unique_id and label
2025-03-02 12:17:46,745 - INFO - INFO --- 18f-mk6240 add unique_id and label
2025-03-02 12:17:46,745 - INFO - {'entities': [{'label': 'disease', 'name': "Alzheimer's disease", 'properties': {'unique_id': 'MESH

In [19]:
kg.chunk[0].context

"Title: The associations between fresh vegetable and fruit consumption and plasma and PET biomarkers in preclinical Alzheimer's disease: A cross-sectional and longitudinal study of Chinese population. Abstract: BACKGROUND: The identification of the modifiable lifestyle factors including dietary habits in older adults of preclinical Alzheimer's disease (AD) and early effective interventions are of great importance. OBJECTIVES: We studied whether the consumption of fresh vegetables and fruits was different between cognitively unimpaired (CU) and cognitively impaired (CI) population and mainly investigated the associations between vegetable and fruit consumption and PET and plasma AD biomarkers in older CU adults with higher beta-amyloid (Abeta) burden. DESIGN, SETTING, AND PARTICIPANTS: Older adults with the age of 50-85 years were enrolled for a cross-sectional and longitudinal study. The groups depended on whether the participants were CU or CI. Partial participants whose habits remain

In [ ]:
# kg.entities[0]

Entity(name=abeta, label=gene, properties=embeddings=array([-6.45760240e-03,  2.40314844e-02, -1.45196152e-01, -2.22277432e-02,
        1.42757736e-02, -2.25719640e-03, -6.24135358e-03,  1.61856960e-03,
       -1.03359260e-03,  6.57131840e-03, -2.32987526e-02,  7.01297100e-02,
        1.11676242e-01, -8.38724940e-03,  6.56710720e-03, -5.94029588e-03,
       -1.08734414e-02, -5.55830352e-03,  3.85745948e-02,  2.17089988e-02,
       -2.11628142e-02, -1.29235102e-02, -1.19219832e-02,  2.56445384e-03,
        1.90849518e-01,  1.27576601e-02,  3.30139200e-03,  3.21932272e-02,
        5.38236436e-03, -2.72192542e-02,  1.62386140e-02, -3.68823472e-02,
        7.38222648e-03, -1.90136200e-03,  3.96177200e-02, -4.14962198e-02,
       -6.07901776e-03,  3.92776136e-02, -3.43964686e-03, -1.60196830e-02,
        4.49002596e-02, -2.03563633e-02, -7.30315984e-02, -2.07667660e-03,
        3.55762458e-02, -1.31322060e-02,  2.86986906e-02,  3.76886758e-02,
        4.70624360e-02, -3.73971217e-02,  3.433

In [16]:
# kg.relationships[0]

In [17]:
GraphIntegrator(uri=URI, username=USERNAME, password=PASSWORD).visualize_graph(knowledge_graph=kg)

2025-03-02 12:18:48,996 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (m))} {position: line: 1, column: 1, offset: 0} for query: 'MATCH (n:disease {name: "alzheimer\'s disease"}), (m:gene {name: "abeta"}) MERGE (n)-[r:is_associated_with]->(m) SET r.embeddings = "0.009747773,-0.018262247,-0.11989937,-0.019365331,-0.0013241308,-0.030844437,-0.010199221,-0.04327022,-0.020864299,